In [1]:
# Objectif: Predire si un film a une moyenne élevée(>=4.0)
# Créer une variable binaire qui prend (1 si averageRating >= 4.0, sinon 0).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
# Chargement du dataset
df=pd.read_csv('/Users/CAMARA110459/Desktop/DOC_JOUR/movie_df.csv',sep=',')
df.head(10)

,movieId,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,directors,writers,averageRating,...,tag1,tag2,directorAvgRating,actor1AvgRating,actor2AvgRating,writerAvgRating,tag0AvgRating,tag1AvgRating,tag2AvgRating,genresAvgRating
0,1,tt0114709,Toy Story,Toy Story,1995,81,"adventure,animation,comedy",nm0005124,"nm0005124,nm0230032,nm0004056,nm0710020,nm0923...",4.15,...,244.0,786.0,3.690000,3.513235,3.275000,3.635714,3.750000,3.457143,4.100000,3.193560
1,2,tt0113497,Jumanji,Jumanji,1995,104,"adventure,comedy,family",nm0002653,"nm0378144,nm0852430,nm0833164,nm0885575",3.55,...,584.0,204.0,3.277778,3.153125,3.277778,3.480000,3.008333,2.861111,3.152174,3.113986
2,3,tt0113228,Grumpier Old Men,Grumpier Old Men,1995,101,"comedy,romance",nm0222043,nm0425756,3.35,...,901.0,902.0,3.115000,3.336842,3.377273,2.875000,2.911765,2.745000,2.761111,3.172996
3,4,tt0114885,Waiting to Exhale,Waiting to Exhale,1995,124,"comedy,drama,romance",nm0001845,"nm0573334,nm0060103",3.00,...,201.0,439.0,2.850000,3.000000,2.983333,2.925000,3.138333,3.105714,3.096552,3.216009
4,5,tt0113041,Father of the Bride Part II,Father of the Bride Part II,1995,106,"comedy,family,romance",nm0796124,"nm0352443,nm0329304,nm0583600,nm0796124",3.05,...,383.0,902.0,3.021429,3.165000,3.403125,3.485714,2.911765,3.162500,2.761111,3.151227
5,6,tt0113277,Heat,Heat,1995,170,"action,crime,drama",nm0000520,nm0000520,4.15,...,498.0,481.0,3.486364,3.498276,3.338235,3.633333,3.462500,3.415385,3.362500,3.184678
6,7,tt0114319,Sabrina,Sabrina,1995,127,"comedy,drama,romance",nm0001628,"nm0853138,nm0000697,nm0499626,nm0070660,nm0713128",3.15,...,864.0,865.0,3.381579,3.374000,2.925000,3.383333,2.798077,3.201111,3.158696,3.216009
7,8,tt0112302,Tom and Huck,Tom and Huck,1995,97,"adventure,comedy,drama",nm0382072,"nm0878494,nm0814085,nm0521739",2.75,...,107.0,1062.0,2.712500,2.900000,3.033333,3.150000,3.008333,3.223684,3.253125,3.178767
8,9,tt0114576,Sudden Death,Sudden Death,1995,111,"action,crime,thriller",nm0001382,"nm0049945,nm0704164",2.90,...,447.0,609.0,2.982353,2.760000,3.100000,2.900000,3.015493,2.967391,3.375000,3.098343
9,10,tt0113189,GoldenEye,GoldenEye,1995,130,"action,adventure,thriller",nm0132709,"nm0001220,nm0289833,nm0128997,nm0270761",3.60,...,2.0,151.0,3.172727,3.200000,3.138889,3.366667,3.083333,3.600000,3.346875,3.070256


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22645 entries, 0 to 22644
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movieId            22645 non-null  int64  
 1   tconst             22645 non-null  object 
 2   primaryTitle       22645 non-null  object 
 3   originalTitle      22645 non-null  object 
 4   startYear          22645 non-null  int64  
 5   runtimeMinutes     22645 non-null  int64  
 6   genres             22645 non-null  object 
 7   directors          22645 non-null  object 
 8   writers            22645 non-null  object 
 9   averageRating      22645 non-null  float64
 10  numVotes           22645 non-null  int64  
 11  actor1             22645 non-null  object 
 12  actor2             22645 non-null  object 
 13  tags_list          5878 non-null   object 
 14  runtimeCat         22645 non-null  object 
 15  wRating            22645 non-null  float64
 16  original_language  226

In [4]:
df.describe()

,movieId,startYear,runtimeMinutes,averageRating,numVotes,wRating,tag0,tag1,tag2,directorAvgRating,actor1AvgRating,actor2AvgRating,writerAvgRating,tag0AvgRating,tag1AvgRating,tag2AvgRating,genresAvgRating
count,22645.000000,22645.000000,22645.000000,22645.000000,2.264500e+04,22645.000000,5878.000000,5878.000000,5878.000000,22645.000000,22645.000000,22645.000000,22645.000000,5878.000000,5878.000000,5878.000000,22645.000000
mean,54567.488585,1987.583175,102.019121,3.177359,3.608553e+04,6.408509,566.728309,579.953896,571.510037,3.177359,3.177359,3.177359,3.177359,3.263687,3.263687,3.263687,3.181314
std,44152.111625,23.663400,20.972202,0.493258,1.206540e+05,0.776583,326.173803,324.658157,321.485742,0.412228,0.401008,0.413136,0.433211,0.326455,0.317875,0.321402,0.122234
min,1.000000,1912.000000,40.000000,0.650000,7.000000e+00,1.647890,1.000000,1.000000,1.000000,0.650000,0.650000,0.650000,0.650000,1.150000,1.730000,1.100000,0.000000
25%,6019.000000,1972.000000,90.000000,2.900000,1.591000e+03,6.011151,276.000000,293.000000,292.000000,2.975000,2.992593,2.991667,2.950000,3.025000,3.037500,3.054167,3.108821
50%,55603.000000,1996.000000,99.000000,3.250000,4.604000e+03,6.415323,575.000000,592.000000,581.000000,3.250000,3.247561,3.225000,3.250000,3.270000,3.281250,3.292262,3.191516
75%,95491.000000,2007.000000,110.000000,3.500000,1.806600e+04,6.886517,862.000000,864.000000,863.000000,3.450000,3.432353,3.445833,3.464286,3.490000,3.475000,3.475000,3.268131
max,131262.000000,2021.000000,442.000000,4.650000,2.970653e+06,9.299009,1128.000000,1128.000000,1128.000000,4.450000,4.450000,4.525000,4.600000,4.450000,4.200000,4.200000,3.570270


In [5]:
df.nunique()

movieId              22631
tconst               22631
primaryTitle         21422
originalTitle        21797
startYear              106
runtimeMinutes         209
genres                 638
directors             9375
writers              19532
averageRating           79
numVotes             13855
actor1                9595
actor2               12103
tags_list             5875
runtimeCat               3
wRating              22042
original_language       79
overview             22188
director              8718
writer               12788
tag0                   708
tag1                   772
tag2                   797
directorAvgRating      896
actor1AvgRating        812
actor2AvgRating        688
writerAvgRating        653
tag0AvgRating          381
tag1AvgRating          413
tag2AvgRating          406
genresAvgRating        638
dtype: int64

In [6]:
df.isnull().sum()


movieId                  0
tconst                   0
primaryTitle             0
originalTitle            0
startYear                0
runtimeMinutes           0
genres                   0
directors                0
writers                  0
averageRating            0
numVotes                 0
actor1                   0
actor2                   0
tags_list            16767
runtimeCat               0
wRating                  0
original_language        0
overview               387
director                 0
writer                   0
tag0                 16767
tag1                 16767
tag2                 16767
directorAvgRating        0
actor1AvgRating          0
actor2AvgRating          0
writerAvgRating          0
tag0AvgRating        16767
tag1AvgRating        16767
tag2AvgRating        16767
genresAvgRating          0
dtype: int64

In [7]:
missing_data = df.isnull().sum() / len(df) * 100

In [8]:
for col in df.select_dtypes(include=['number']).columns:  # Sélectionne uniquement les colonnes numériques
        df[col].fillna(df[col].mean(), inplace=True)

/var/folders/h0/shsgny_92p3dxs5yjl2tz2z00000gp/T/ipykernel_47290/711223711.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [9]:
df['tags_list'] = df['tags_list'].fillna(df['tags_list'].mode()[0])

In [29]:
df['tags_list'] .unique()

array(['1036,244,786,589,64,588,785,204,186,536',
       '29,584,204,588,951,377,62,415,203,374',
       '451,901,902,230,742,264,299,464,863,397', ...,
       '733,689,1051,994,1025,682,999,268,558,911',
       '596,798,799,800,258,878,877,806,472,402',
       '169,146,625,168,638,587,145,493,727,742'], dtype=object)

In [10]:
df.isnull().sum() / len(df) * 100

movieId              0.000000
tconst               0.000000
primaryTitle         0.000000
originalTitle        0.000000
startYear            0.000000
runtimeMinutes       0.000000
genres               0.000000
directors            0.000000
writers              0.000000
averageRating        0.000000
numVotes             0.000000
actor1               0.000000
actor2               0.000000
tags_list            0.000000
runtimeCat           0.000000
wRating              0.000000
original_language    0.000000
overview             1.708987
director             0.000000
writer               0.000000
tag0                 0.000000
tag1                 0.000000
tag2                 0.000000
directorAvgRating    0.000000
actor1AvgRating      0.000000
actor2AvgRating      0.000000
writerAvgRating      0.000000
tag0AvgRating        0.000000
tag1AvgRating        0.000000
tag2AvgRating        0.000000
genresAvgRating      0.000000
dtype: float64

In [11]:
df.dropna(subset=['overview'], inplace=True)

In [12]:
df.isnull().sum() / len(df) * 100

movieId              0.0
tconst               0.0
primaryTitle         0.0
originalTitle        0.0
startYear            0.0
runtimeMinutes       0.0
genres               0.0
directors            0.0
writers              0.0
averageRating        0.0
numVotes             0.0
actor1               0.0
actor2               0.0
tags_list            0.0
runtimeCat           0.0
wRating              0.0
original_language    0.0
overview             0.0
director             0.0
writer               0.0
tag0                 0.0
tag1                 0.0
tag2                 0.0
directorAvgRating    0.0
actor1AvgRating      0.0
actor2AvgRating      0.0
writerAvgRating      0.0
tag0AvgRating        0.0
tag1AvgRating        0.0
tag2AvgRating        0.0
genresAvgRating      0.0
dtype: float64

In [13]:
# Création de la variable cible
df['high_rating'] = (df['averageRating'] >= 4.0).astype(int)

In [14]:
# Vérification du déséquilibre des classes
print(df['high_rating'].value_counts(normalize=True)*100)

high_rating
0    97.443616
1     2.556384
Name: proportion, dtype: float64


In [ ]:
# Sélection des caractéristiques
numerical_features = [
    'runtimeMinutes', 'numVotes', 
    'directorAvgRating', 'actor1AvgRating', 
    'actor2AvgRating', 'writerAvgRating',
    'tag0AvgRating', 'tag1AvgRating', 'tag2AvgRating'
]

categorical_features = ['original_language', 'runtimeCat']

In [23]:
# Prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [24]:
X = preprocessor.fit_transform(df)
y = df['high_rating']

In [25]:
# Division stratifiée train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [26]:
# Application du SMOTE 
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [27]:
# Entraînement avec validation croisée stratifiée
model = RandomForestClassifier(class_weight='balanced', random_state=42)

skf = StratifiedKFold(n_splits=5)
for train_idx, val_idx in skf.split(X_train_res, y_train_res):
    X_fold_train, y_fold_train = X_train_res[train_idx], y_train_res[train_idx]
    X_val, y_val = X_train_res[val_idx], y_train_res[val_idx]
    
    model.fit(X_fold_train, y_fold_train)
    val_pred = model.predict(X_val)
    print("Validation Fold Report:")
    print(classification_report(y_val, val_pred))

Validation Fold Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      3471
           1       0.99      0.97      0.98      3470

    accuracy                           0.98      6941
   macro avg       0.98      0.98      0.98      6941
weighted avg       0.98      0.98      0.98      6941

Validation Fold Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3470
           1       0.99      1.00      0.99      3471

    accuracy                           0.99      6941
   macro avg       0.99      0.99      0.99      6941
weighted avg       0.99      0.99      0.99      6941

Validation Fold Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3470
           1       0.99      1.00      0.99      3470

    accuracy                           0.99      6940
   macro avg       0.99      0.99      0.99      6940
we

In [28]:
# Évaluation finale sur le test set
test_pred = model.predict(X_test)
print("\nTest Set Report:")
print(classification_report(y_test, test_pred))
print("Matrice de Confusion:")
print(confusion_matrix(y_test, test_pred))


Test Set Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4338
           1       0.63      0.68      0.66       114

    accuracy                           0.98      4452
   macro avg       0.81      0.84      0.82      4452
weighted avg       0.98      0.98      0.98      4452

Matrice de Confusion:
[[4292   46]
 [  36   78]]
